## Setup and installs

In [ ]:
# Install into current kernel 
%pip install --quiet --upgrade pip
%pip install --quiet pandas numpy patsy cmdstanpy arviz xarray netcdf4

# Ensure project root
import os
os.chdir('/Users/ogeohia/PYTHON/eo-colon-cancer-trends-ci5plus')
print("CWD:", os.getcwd())
print("Python:", sys.executable)
print("CmdStan path:", csp.cmdstan_path() or "Not installed")

# CmdStan toolchain installation
csp.install_cmdstan()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/Users/ogeohia/anaconda3/envs/colon-cancer-data/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python: /Users/ogeohia/anaconda3/envs/colon-cancer-data/bin/python
CmdStan path: /Users/ogeohia/anaconda3/envs/colon-cancer-data/bin/cmdstan
CmdStan install directory: /Users/ogeohia/.cmdstan
Installing CmdStan version: 2.37.0
Download successful, file: /var/folders/sv/358t1yp13rld61ff_m2ty5p40000gn/T/tmpmn9n6sbh
Extracting distribution
Unpacked download as cmdstan-2.37.0
Building version cmdstan-2.37.0, may take several minutes, depending on your system.
Installed cmdstan-2.37.0
Test model compilation


True

## Load data and build Stan inputs

In [4]:
df = pd.read_csv("../data/colon_cancer_full.csv")
assert (df['py'] > 0).all(), "Found zero/negative py values."

# Age spline (df=4 as in your GLM)
age_spline = dmatrix("bs(age_cont, df=4, include_intercept=False)", data=df, return_type='dataframe')
B_age = np.asarray(age_spline)
K_age = B_age.shape[1]

# Indicators and IDs
df['male_ind'] = (df['sex_label'] == 'Male').astype(int)

country_codes = {c:i+1 for i,c in enumerate(sorted(df['country'].unique()))}
region_codes  = {r:i+1 for i,r in enumerate(sorted(df['region'].unique()))}

country_id = df['country'].map(country_codes).astype(int).to_numpy()
J_country = len(country_codes)

# Country -> region mapping
country_to_region = df.groupby('country')['region'].first().to_dict()
region_id_country = np.array(
    [region_codes[country_to_region[c]] for c,_i in sorted(country_codes.items(), key=lambda x: x[1])],
    dtype=int
)
R_region = len(region_codes)

year_c = (df['year'] - df['year'].mean()).to_numpy()

stan_data = {
    'N': len(df),
    'y': df['cases'].astype(int).to_numpy(),
    'py': df['py'].to_numpy(),
    'J_country': J_country,
    'R_region': R_region,
    'country_id': country_id,
    'region_id_country': region_id_country,
    'K_age': K_age,
    'B_age': B_age,
    'male': df['male_ind'].astype(int).to_numpy(),
    'year_c': year_c
}
len(df), K_age, J_country, R_region

(92326, 5, 48, 12)

## Write Stan model 

In [8]:
stan_src = r"""
data {
  int<lower=1> N;
  array[N] int<lower=0> y;
  vector<lower=0>[N] py;
  int<lower=1> J_country;
  int<lower=1> R_region;
  array[N] int<lower=1> country_id;
  array[J_country] int<lower=1> region_id_country;
  int<lower=1> K_age;
  matrix[N, K_age] B_age;
  array[N] int<lower=0, upper=1> male;
  vector[N] year_c;
}
parameters {
  real alpha;
  vector[R_region] z_region;
  real<lower=0> sigma_region;
  vector[J_country] z_country;
  real<lower=0> sigma_country;
  vector[K_age] beta_age;
  real beta_male;
  real beta_year;
  real<lower=0> phi;
}
transformed parameters {
  vector[R_region] region_eff = z_region * sigma_region;
  vector[J_country] country_eff;
  for (j in 1:J_country)
    country_eff[j] = region_eff[ region_id_country[j] ] + z_country[j] * sigma_country;
}
model {
  alpha        ~ normal(0, 2);
  z_region     ~ normal(0, 1);
  sigma_region ~ exponential(1);
  z_country    ~ normal(0, 1);
  sigma_country~ exponential(1);
  beta_age     ~ normal(0, 1);
  beta_male    ~ normal(0, 1);
  beta_year    ~ normal(0, 0.5);
  phi          ~ exponential(1);

  for (n in 1:N) {
    real eta = alpha
               + country_eff[country_id[n]]
               + B_age[n] * beta_age
               + beta_male * male[n]
               + beta_year * year_c[n]
               + log(py[n]);
    y[n] ~ neg_binomial_2_log(eta, phi);
  }
}
generated quantities {
  vector[N] log_lambda;
  array[N] int y_rep;
  for (n in 1:N) {
    real eta = alpha
               + country_eff[country_id[n]]
               + B_age[n] * beta_age
               + beta_male * male[n]
               + beta_year * year_c[n]
               + log(py[n]);
    log_lambda[n] = eta;
    y_rep[n] = neg_binomial_2_log_rng(eta, phi);
  }
}
"""
os.makedirs("models", exist_ok=True)
stan_path = "models/hierarchical_colon_nb.stan"
with open(stan_path, "w", encoding="utf-8") as f:
    f.write(stan_src)
print("Wrote:", stan_path)

Wrote: models/hierarchical_colon_nb.stan


In [ ]:
model = CmdStanModel(stan_file="models/hierarchical_colon_nb.stan", force_compile=True)

fit = model.sample(
    data=stan_data,
    chains=4, parallel_chains=4,
    iter_warmup=1000, iter_sampling=1000,
    adapt_delta=0.95, max_treedepth=12
)

print(fit)
fit.summary().loc[['alpha','sigma_region','sigma_country','beta_male','beta_year','phi']]

18:47:32 - cmdstanpy - INFO - compiling stan file /Users/ogeohia/PYTHON/eo-colon-cancer-trends-ci5plus/notebooks/models/hierarchical_colon_nb.stan to exe file /Users/ogeohia/PYTHON/eo-colon-cancer-trends-ci5plus/notebooks/models/hierarchical_colon_nb
18:47:52 - cmdstanpy - INFO - compiled model executable: /Users/ogeohia/PYTHON/eo-colon-cancer-trends-ci5plus/notebooks/models/hierarchical_colon_nb
18:47:55 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status





chain 1 |▍         | 00:01 Status

chain 1 |▉         | 02:21 Iteration:    1 / 2000 [  0%]  (Warmup)
